In [3]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [4]:
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Weaviate
from typing import List
import os
import weaviate

def openai_chat_client():
  deployment = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_ID")
  key = os.getenv("AZURE_OPENAI_API_KEY")
  resource = os.getenv("AZURE_OPENAI_RESOURCE_NAME")

  return AzureChatOpenAI(deployment_name=deployment, 
                         openai_api_key=key, 
                         openai_api_base=f"https://{resource}.openai.azure.com/",
                         openai_api_version="2023-03-15-preview")


def weaviate_vector_store(index_name: str, text_key: str, attributes: List[str] = []):
  weaviate_url = os.environ['WEAVIATE_URL']
  weaviate_api_key = os.environ['WEAVIATE_API_KEY']
  openai_api_key = os.environ['AZURE_OPENAI_API_KEY']

  auth_config = weaviate.AuthApiKey(api_key=weaviate_api_key)

  client = weaviate.Client(
      url=weaviate_url,
      auth_client_secret=auth_config,
      additional_headers={
          "X-OpenAI-Api-Key": openai_api_key
      }
  )
  return Weaviate(client=client, 
                  index_name=index_name, 
                  text_key=text_key, 
                  attributes=attributes)

In [5]:
import base64
import json
import os
from langchain.chains import RetrievalQAWithSourcesChain


from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)


def ask(question):
  index_name = "Work"
  text_key = "title"
  attributes = ["identifier", "title", "source", "alternate_title", "contributor", "create_date", "creator", "date_created", "description", "genre", "keywords", "language", "location", "physical_description_material", "physical_description_size", "scope_and_contents", "style_period", "subject", "table_of_contents", "technique"]

  weaviate = weaviate_vector_store(
    index_name=index_name, 
    text_key=text_key, 
    attributes=attributes
  )
  
  client = openai_chat_client()
  chain = RetrievalQAWithSourcesChain.from_chain_type(
    client, 
    chain_type="stuff", 
    retriever=weaviate.as_retriever()
  )

  return chain({"question": question})

In [19]:
ask("Where is Northwestern University located?")

{'question': 'Where is Northwestern University located?',
 'answer': 'Northwestern University is located in Evanston, Illinois.\n',
 'sources': 'https://dc.library.northwestern.edu/items/e353efcd-8664-4ede-9f0b-1f8d35aaefa0, https://dc.library.northwestern.edu/items/52de7487-eb47-4b22-bf57-59a049e7fc09, https://dc.library.northwestern.edu/items/f08996ed-9125-430f-a1c2-6c21c7527772, https://dc.library.northwestern.edu/items/aa23199b-782b-41c3-9cb3-ac14da315686'}

In [20]:
ask("Is Northwestern near Lake Michigan?")

{'question': 'Is Northwestern near Lake Michigan?',
 'answer': 'Yes, Northwestern is near Lake Michigan. \n',
 'sources': 'Pictures in links provided show views of Lake Michigan from Northwestern University and locations near it.'}

In [24]:
ask("Does the collection have any images of lions in the wild?")

{'question': 'Does the collection have any images of lions in the wild?',
 'answer': 'Yes, the collection has at least one image of lions in the wild in East Africa.\n',
 'sources': 'https://dc.library.northwestern.edu/items/625c6477-93b9-4ada-bc4f-8a3f0b63dee0'}

In [21]:
ask("Can you show me examples of war propaganda?")

{'question': 'Can you show me examples of war propaganda?',
 'answer': 'Examples of war propaganda include "I\'m counting on you! : don\'t discuss troop movements, ship sailings, war equipment," "United : the United Nations fight for freedom," and "Bombers\' guardians : your planes carry the fight to the Axis." \n',
 'sources': 'https://dc.library.northwestern.edu/items/5bdea09c-0fd2-4297-b58c-3ff2df3ba73f, https://dc.library.northwestern.edu/items/c52a9fea-3bf8-4edd-a7f9-5c090e092b9b, https://dc.library.northwestern.edu/items/a8d6c689-7ae9-4942-9b71-5bad6fe1095f, https://dc.library.northwestern.edu/items/3f3edf71-f9d2-4361-88a3-3de082ebdcd9'}

In [23]:
ask("Can you show me historical Northwestern football matches?")

{'question': 'Can you show me historical Northwestern football matches?',
 'answer': 'Yes, historical Northwestern football matches are available at the following links: \n- https://dc.library.northwestern.edu/items/57dcb664-7878-490e-87b6-6c4283b1854e (Preparations, 1943)\n- https://dc.library.northwestern.edu/items/0bd480f3-2503-4e0f-ae9e-9662592567fe (vs. Notre Dame, 1944)\n- https://dc.library.northwestern.edu/items/b0ae1256-efa5-4dc7-b54e-71029b2c5495 (vs. Wisconsin, 1939)\n- https://dc.library.northwestern.edu/items/c0ef3eba-d71b-4851-99c0-a65cf52c133d (vs. Wisconsin, 1940)\n',
 'sources': 'Provided links'}

In [22]:
ask("Who did John Cage correspond with during his life?")

{'question': 'Who did John Cage correspond with during his life?',
 'answer': 'John Cage corresponded with H.M.L., Barbara Chandler, Marion Swartz, and George Avakian.\n',
 'sources': 'https://dc.library.northwestern.edu/items/e61667ee-0c20-418b-84ec-ec6200cca270, https://dc.library.northwestern.edu/items/33906799-3fc3-4f1b-834a-5c78c7ecea76, https://dc.library.northwestern.edu/items/0a2308e6-93a8-40a9-b5ed-77f8f2ad7697, https://dc.library.northwestern.edu/items/816c7d2f-72e5-425c-8bf2-5e96e629f866'}

In [25]:
ask("Clarinet is my favorite, any examples?")

{'question': 'Clarinet is my favorite, any examples?',
 'answer': 'There is an example of a Trio in B-flat Major Op. 11 for Clarinet, Cello and Piano.\n',
 'sources': 'https://dc.library.northwestern.edu/items/50043b3c-7e35-40b9-ac0b-2107da4287ce'}